# Analyze A in Pareto

In [14]:
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from bokeh.layouts import column, row
from bokeh.models import Range1d
from bokeh.io import export_png
output_notebook()

import numpy as np

from voltagebudget import util
from voltagebudget.util import locate_firsts
from voltagebudget.util import locate_peaks
from voltagebudget.util import filter_voltages
from voltagebudget.util import read_modes
from voltagebudget.util import read_stim
from voltagebudget.util import mad
from voltagebudget.neurons import adex

# Temporal params
time_step = 1e-5
coincidence_t = 1e-3
seed_value = 42

Loading BokehJS ...

## Exp 32

In [15]:
f = 30
N = 50
t = 0.4

results = np.loadtxt("../data/exp32_d-0.2e-3_f{}.csv".format(f), delimiter=",", skiprows=1)

As = results[:, 0]

In [16]:
stim = "../data/stim1.csv"
mode = "regular"
noise = False

# Just choose
E = .21772

# Find the phase begin a osc cycle at E 
phi_E = float(-E * 2 * np.pi * f)

# -
params, w_in, bias_in, sigma = read_modes(mode)
if not noise:
    sigma = 0

stim_data = read_stim(stim)
ns = np.asarray(stim_data['ns'])
ts = np.asarray(stim_data['ts'])

spike_sets = []
for A in As:
    print(">>> Running A {}".format(A))
    
    ns_a, ts_a, _ = adex(
        N,
        t,
        ns,
        ts,
        w_in=w_in,
        bias_in=bias_in,
        f=f,
        A=A,
        phi=phi_E,
        sigma=sigma,
        budget=True,
        seed_value=seed_value,
        time_step=time_step,
        save_args=None,
        **params)
    
    spike_sets.append([A, ns_a.copy(), ts_a.copy()])

>>> Running A 4.57722038188e-11
>>> Running A 3.93712882988e-11
>>> Running A 4.2188618262e-11
>>> Running A 4.90674430069e-11
>>> Running A 4.62226968402e-11
>>> Running A 4.25917521753e-11
>>> Running A 3.0478147474e-11
>>> Running A 3.77783089441e-11
>>> Running A 3.78659720244e-11
>>> Running A 2.1126088678e-11
>>> Running A 1.02862106213e-11
>>> Running A 5.85296157968e-13
>>> Running A 2.36934486872e-11
>>> Running A 4.43212761658e-11
>>> Running A 6.00852099809e-12
>>> Running A 3.7986271936e-11
>>> Running A 3.32796118932e-11
>>> Running A 1.4412547823e-11
>>> Running A 4.23552042527e-11
>>> Running A 4.07561521719e-11
>>> Running A 2.46349135262e-12
>>> Running A 2.40856533251e-11
>>> Running A 1.39315080106e-11
>>> Running A 8.70330397737e-12
>>> Running A 3.7677008279e-11
>>> Running A 4.34182453145e-11
>>> Running A 1.27023998358e-11
>>> Running A 3.18275996421e-11
>>> Running A 9.62323557809e-12
>>> Running A 3.38700418437e-11
>>> Running A 2.66081239983e-11
>>> Running A 

In [17]:
from operator import itemgetter
spike_sets = sorted(spike_sets, key=itemgetter(0))

for A, ns_a, ts_a in spike_sets:
    p = figure(plot_width=300, plot_height=200)
    p.circle(ts_a, ns_a, color="black", alpha=0.3, legend="A: {:.2f} mad: {:.4f}".format(1e12*A, mad(ts_a)))
    p.xaxis.axis_label = 'Time (s)'
    p.yaxis.axis_label = 'N'
    p.x_range = Range1d(0, t)
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    show(p)